In [1]:
import pandas as pd
from datetime import datetime 
import plotly.graph_objects as go
from pandas import DataFrame

In [19]:
# Import data we gathered from Bitmex
price_history = pd.read_csv('History_BTC/BTC_history_5m.csv')
#Set timestamo as index
price_history =price_history.set_index('timestamp')

#Here we will create custom indicators that will be sell and buy signals 
#Our buy signal will be average range of previuous 10 candles minus previous close
#And sell signal will be average range of previuous 10 candles pluss previous close
price_history['candle_range'] = price_history['high'] - price_history['low']
price_history=price_history.iloc[::-1]
price_history['AverageRange']= round(price_history['candle_range'].rolling(window =10).mean() *2)/2
price_history['AR_top']=price_history['close']+price_history['AverageRange']
price_history['AR_bot']=price_history['close']-price_history['AverageRange']
price_history.head(15)

,high,low,close,open,candle_range,AverageRange,AR_top,AR_bot
timestamp,,,,,,,,
2017-05-30T10:15:00.000Z,2307.4,2303.5,2307.1,2305.4,3.9,NaN,NaN,NaN
2017-05-30T10:20:00.000Z,2310.4,2306.9,2310.4,2307.1,3.5,NaN,NaN,NaN
2017-05-30T10:25:00.000Z,2312.0,2310.5,2312.0,2310.4,1.5,NaN,NaN,NaN
2017-05-30T10:30:00.000Z,2318.8,2312.0,2316.3,2312.0,6.8,NaN,NaN,NaN
2017-05-30T10:35:00.000Z,2318.1,2315.3,2318.1,2316.3,2.8,NaN,NaN,NaN
2017-05-30T10:40:00.000Z,2330.0,2318.0,2328.4,2318.1,12.0,NaN,NaN,NaN
2017-05-30T10:45:00.000Z,2328.2,2323.9,2325.2,2328.4,4.3,NaN,NaN,NaN
2017-05-30T10:50:00.000Z,2326.1,2323.9,2325.9,2325.2,2.2,NaN,NaN,NaN
2017-05-30T10:55:00.000Z,2329.5,2325.0,2327.4,2325.9,4.5,NaN,NaN,NaN


In [20]:
# will plot only last 150  items to show results
price_history2=price_history.tail(250)

In [21]:
Candle = go.Candlestick(x=price_history2.index,
                open=price_history2['open'],
                high=price_history2['high'],
                low=price_history2['low'],
                close=price_history2['close'],name="Price")

Trace = go.Scatter(x=price_history2.index,
                   y=price_history2['AR_top'],mode="markers",marker=dict(color="red"),name="Sell signals")
Trace2 = go.Scatter(x=price_history2.index,
                   y=price_history2['AR_bot'],mode="markers",marker=dict(color="green"),name="Buy signals")

fig =go.Figure([Candle, Trace,Trace2])

fig.update_layout(xaxis_rangeslider_visible=False)

# green dots are buy signals and red sell are buy signals

In [22]:
# this big function simulates trading with our buy and sell signals by iterating every row from begining

def backtest(data,ar_days):
    capital=1000
    trades=[]
    buy_signal = data.iloc[ar_days]['AR_bot']
    sell_signal = data.iloc[ar_days]['AR_top']
    take_profit = 0
    stop_loss = 0
    trade_buy = 0
    trade_sell = 0
    tradeOn_sell = False
    tradeOn_buy = False
    position = 0
    entrydate=0
    tradeSkip=False
    for index, row in data.iloc[ar_days:].iterrows():
        
        while tradeOn_sell:
            if row['high'] >= stop_loss and row['low']<= take_profit:
                trades.append([entrydate,index,"","","",capital,"",'Whiplash'])
                tradeOn_sell=False
                sell_signal = row['AR_top']
                buy_signal = row['AR_bot']
                tradeSkip=True
                break
            if row['high'] >= stop_loss:
                capital -= position*(stop_loss/trade_sell-1)
                trades.append([entrydate,index,trade_sell,stop_loss,position*-(stop_loss/trade_sell-1),capital,"SELL","LOSS"])
                tradeOn_sell=False
                sell_signal = row['AR_top']
                buy_signal = row['AR_bot']
                tradeSkip=True
                break
            if row['low'] <= take_profit:
                capital -= position*(take_profit/trade_sell-1)
                trades.append([entrydate,index,trade_sell,take_profit,position*-(take_profit/trade_sell-1),capital,"SELL",'WIN'])
                tradeOn_sell=False
                sell_signal = row['AR_top']
                buy_signal = row['AR_bot']
                tradeSkip=True
                break
            break
            
        while tradeOn_buy:
            if row['low'] <= stop_loss and row['high']>= take_profit:
                trades.append([entrydate,index,"","","",capital,"",'Whiplash'])
                tradeOn_buy=False
                sell_signal = row['AR_top']
                buy_signal = row['AR_bot']
                tradeSkip=True
                break
            if row['low'] <= stop_loss:
                capital -= position*-(stop_loss/trade_buy-1)
                trades.append([entrydate,index,trade_buy,stop_loss,position*(stop_loss/trade_buy-1),capital,"BUY","LOSS"])
                tradeOn_buy=False
                sell_signal = row['AR_top']
                buy_signal = row['AR_bot']
                tradeSkip=True
                break
            if row['high'] >= take_profit:
                capital += position*(take_profit/trade_buy-1)
                trades.append([entrydate,index,trade_buy,take_profit,position*(take_profit/trade_buy-1),capital,"BUY","WIN"])
                tradeOn_buy=False
                sell_signal = row['AR_top']
                buy_signal = row['AR_bot']
                tradeSkip=True
                break
            break
            
        while not tradeOn_buy and not tradeOn_sell and not tradeSkip:
            if row['high']>=sell_signal and row['low']<=buy_signal:
                trades.append([index,"","","","",capital,"",'Whiplash'])
                sell_signal = row['AR_top']
                buy_signal = row['AR_bot']
                break
            if row['high'] >= sell_signal:
                trade_sell=sell_signal
                stop_loss = sell_signal + (row['AverageRange'])
                take_profit = sell_signal - (row['AverageRange'])
                position = capital*0.01
                entrydate = index
                tradeOn_sell = True
                if row['high'] >= stop_loss and row['low']<= take_profit:
                    trades.append([entrydate,index,"","","",capital,"",'Whiplash'])
                    tradeOn_sell=False
                    sell_signal = row['AR_top']
                    buy_signal = row['AR_bot']
                    break
                if row['high'] >= stop_loss:
                    capital -= position*(stop_loss/trade_sell-1)
                    trades.append([entrydate,index,trade_sell,stop_loss,position*-(stop_loss/trade_sell-1),capital,"SELL","LOSS"])
                    tradeOn_sell=False
                    sell_signal = row['AR_top']
                    buy_signal = row['AR_bot']
                    break
                if row['low'] <= take_profit:
                    if row['close']<=take_profit:
                        capital -= position*(stop_loss/trade_sell-1)
                        trades.append([entrydate,index,trade_sell,take_profit,position*-(stop_loss/trade_sell-1),capital,"SELL","LOSS"])
                        tradeOn_sell=False
                        sell_signal = row['AR_top']
                        buy_signal = row['AR_bot']
                        break
                break
            if row['low'] <= buy_signal:
                trade_buy = buy_signal
                stop_loss = trade_buy - (row['AverageRange'])
                take_profit = trade_buy + (row['AverageRange'])
                position = capital*0.01
                entrydate = index
                tradeOn_buy = True
                if row['low'] <= stop_loss and row['high']>= take_profit:
                    trades.append([entrydate,index,"","","",capital,"",'Whiplash'])
                    tradeOn_buy=False
                    sell_signal = row['AR_top']
                    buy_signal = row['AR_bot']
                    break
                if row['low'] <= stop_loss:
                    capital -= position*-(stop_loss/trade_buy-1)
                    trades.append([entrydate,index,trade_buy,stop_loss,position*(stop_loss/trade_buy-1),capital,"BUY","LOSS"])
                    tradeOn_buy=False
                    sell_signal = row['AR_top']
                    buy_signal = row['AR_bot']
                    break
                if row['high'] >= take_profit:
                    if take_profit <= row['close']:
                        capital += position*(take_profit/trade_buy-1)
                        trades.append([entrydate,index,trade_buy,take_profit,position*(take_profit/trade_buy-1),capital,"BUY","WIN"])
                        tradeOn_buy=False
                        sell_signal = row['AR_top']
                        buy_signal = row['AR_bot']
                        break
                break
            sell_signal=row['AR_top']
            buy_signal=row['AR_bot']
            break
        tradeSkip=False
    return trades

In [23]:
# Results are saved in a list which is in another big list
backtest_results = backtest(price_history,10)
backtest_results[:3]

[['2017-05-30T11:05:00.000Z',
  '2017-05-30T11:05:00.000Z',
  2311.9,
  2318.4,
  0.02811540291535186,
  1000.0281154029153,
  'BUY',
  'WIN'],
 ['2017-05-30T11:10:00.000Z',
  '2017-05-30T11:15:00.000Z',
  2311.9,
  2319.4,
  0.032441761605267086,
  1000.0605571645206,
  'BUY',
  'WIN'],
 ['2017-05-30T11:20:00.000Z',
  '2017-05-30T11:45:00.000Z',
  2313.3,
  2322.3,
  0.03890781573717555,
  1000.0994649802577,
  'BUY',
  'WIN']]

In [24]:
#Convert list to panda dataframe
results=DataFrame.from_records(backtest_results,columns=("entrydate","exitdate","entryprice",'exitprice','profit','capital','tradetype','win_loss'))
results.tail()

,entrydate,exitdate,entryprice,exitprice,profit,capital,tradetype,win_loss
49472,2019-10-09T14:25:00.000Z,2019-10-09T14:25:00.000Z,8410.5,8449.5,-0.0417844,901.053920,SELL,LOSS
49473,2019-10-09T14:50:00.000Z,2019-10-09T15:20:00.000Z,8461,8511.5,-0.05378,901.000141,SELL,LOSS
49474,2019-10-09T15:25:00.000Z,2019-10-09T15:30:00.000Z,8547,8588,-0.043221,900.956919,SELL,LOSS
49475,2019-10-09T15:35:00.000Z,2019-10-09T16:35:00.000Z,8570.5,8623,0.0551896,901.012109,BUY,WIN
49476,2019-10-09T16:40:00.000Z,2019-10-09T16:45:00.000Z,8639,8613,0.0271169,901.039226,SELL,WIN


In [25]:
def show_results(results):
    end_capital=list(results['capital'])[-1]
    total_trades= results.shape[0]

    if end_capital>1000:
        print(f"End result is {round(end_capital,2)}, gain of {round(end_capital-1000,2)} ({round((end_capital/1000-1)*100,2)}%)")
    else:
        print(f"End result is {round(end_capital,2)}, loss of {-round(end_capital-1000,2)} ({-round((end_capital/1000-1)*100,2)}%)")

    whiplash=results[results['win_loss']=="Whiplash"].shape[0]

    total_buys=results[results['tradetype']=='BUY']
    total_buys_count=total_buys.shape[0]
    total_buys_loss=total_buys[total_buys['win_loss']=='WIN'].shape[0]
    total_buys_win=total_buys[total_buys['win_loss']=='LOSS'].shape[0]

    total_sells=results[results['tradetype']=='SELL']
    total_sells_count=total_sells.shape[0]
    total_sells_win=total_sells[total_sells['win_loss']=='WIN'].shape[0]
    total_sells_loss=total_sells[total_sells['win_loss']=='LOSS'].shape[0]
    print("")
    print(f"Total trade count is {total_trades}")
    print(f"WIN trades - {total_buys_win+total_sells_win}({round((total_buys_win+total_sells_win)/total_trades*100,2)}%)")
    print(f'    Buy trades - {total_buys_win} ({round(total_buys_win/(total_buys_win+total_sells_win)*100,2)}%)') 
    print(f'    Sell trades - {total_sells_win} ({round(total_sells_win/(total_buys_win+total_sells_win)*100,2)}%)') 
    print(f"LOSS trades - {total_buys_loss+total_sells_loss}({round((total_buys_loss+total_sells_loss)/total_trades*100,2)}%)")
    print(f'    Buy trades - {total_buys_loss} ({round(total_buys_loss/(total_buys_loss+total_sells_loss)*100,2)}%)') 
    print(f'    Sell trades - {total_sells_loss} ({round(total_sells_loss/(total_buys_loss+total_sells_loss)*100,2)}%)') 
    print("")
    print(f'Whiplash trades - {whiplash} ({round(whiplash/total_trades,4)*100}%)')
    print("")
    longest_wins=0
    wins_count=0
    for i,row in results.iterrows():
        if row['win_loss']=="WIN":
            wins_count+=1
        else:
            if wins_count>longest_wins:
                longest_wins=wins_count
            wins_count=0
    print(f'Longest Wining streak: {longest_wins}')
    longest_loss=0
    loss_count=0
    for i,row in results.iterrows():
        if row['win_loss']=="LOSS":
            loss_count+=1
        else:
            if loss_count>longest_loss:
                longest_loss=loss_count
            loss_count=0
    print(f'Longest Losing streak: {longest_loss}')

In [26]:
show_results(results)

End result is 901.04, loss of 98.96 (9.9%)

Total trade count is 49477
WIN trades - 21126(42.7%)
    Buy trades - 12436 (58.87%)
    Sell trades - 8690 (41.13%)
LOSS trades - 23134(46.76%)
    Buy trades - 10122 (43.75%)
    Sell trades - 13012 (56.25%)

Whiplash trades - 5217 (10.54%)

Longest Wining streak: 25
Longest Losing streak: 20


In [27]:
# Lest plot latest entries and exits on graph
entries = results[['entrydate','entryprice']].iloc[-50:]
entries =entries.set_index('entrydate')
exits = results[['exitdate','exitprice']].iloc[-50:]
exits =exits.set_index('exitdate')
exits.head()

,exitprice
exitdate,
2019-10-08T23:50:00.000Z,8201
2019-10-09T00:00:00.000Z,8189
2019-10-09T00:20:00.000Z,8206
2019-10-09T00:35:00.000Z,
2019-10-09T01:35:00.000Z,8186.5


In [28]:
Candle = go.Candlestick(x=price_history2.index,
                open=price_history2['open'],
                high=price_history2['high'],
                low=price_history2['low'],
                close=price_history2['close'],name="Price")

Trace = go.Scatter(x=price_history2.index,
                   y=price_history2['AR_top'],mode="markers",marker=dict(color="red"),name="Sell signals")
Trace2 = go.Scatter(x=price_history2.index,
                   y=price_history2['AR_bot'],mode="markers",marker=dict(color="green"),name="Buy signals")
Entries = go.Scatter(x=entries.index,
                   y=entries['entryprice'],mode="markers",marker=dict(color="blue"), name="Entries")
Exits = go.Scatter(x=exits.index,
                   y=exits['exitprice'],mode="markers",marker=dict(color="purple"),name="Exits")


fig =go.Figure([Candle,Trace,Trace2,Entries,Exits])

fig.update_layout(xaxis_rangeslider_visible=False)

# Here you can see blue dots that are entries and purple dots that are exits. By visualasing entries and exits we can chack if our function correctly simulated trading
# Entries are one candle in front of signals, because signal is generated looking on previous candles.

In [17]:
def show_results(results):
    "Print out statistic from your backtest results"
    end_capital=list(results['capital'])[-1]
    total_trades= results.shape[0]

    if end_capital>1000:
        print(f"End result is {round(end_capital,2)}, gain of {round(end_capital-1000,2)} ({round((end_capital/1000-1)*100,2)}%)")
    else:
        print(f"End result is {round(end_capital,2)}, loss of {-round(end_capital-1000,2)} ({-round((end_capital/1000-1)*100,2)}%)")

    whiplash=results[results['win_loss']=="Whiplash"].shape[0]

    total_buys=results[results['tradetype']=='BUY']
    total_buys_count=total_buys.shape[0]
    total_buys_loss=total_buys[total_buys['win_loss']=='WIN'].shape[0]
    total_buys_win=total_buys[total_buys['win_loss']=='LOSS'].shape[0]

    total_sells=results[results['tradetype']=='SELL']
    total_sells_count=total_sells.shape[0]
    total_sells_win=total_sells[total_sells['win_loss']=='WIN'].shape[0]
    total_sells_loss=total_sells[total_sells['win_loss']=='LOSS'].shape[0]
    print("")
    print(f"Total trade count is {total_trades}")
    print(f"WIN trades - {total_buys_win+total_sells_win}({round((total_buys_win+total_sells_win)/total_trades*100,2)}%)")
    print(f'    Buy trades - {total_buys_win} ({round(total_buys_win/(total_buys_win+total_sells_win)*100,2)}%)') 
    print(f'    Sell trades - {total_sells_win} ({round(total_sells_win/(total_buys_win+total_sells_win)*100,2)}%)') 
    print(f"LOSS trades - {total_buys_loss+total_sells_loss}({round((total_buys_loss+total_sells_loss)/total_trades*100,2)}%)")
    print(f'    Buy trades - {total_buys_loss} ({round(total_buys_loss/(total_buys_loss+total_sells_loss)*100,2)}%)') 
    print(f'    Sell trades - {total_sells_loss} ({round(total_sells_loss/(total_buys_loss+total_sells_loss)*100,2)}%)') 
    print("")
    print(f'Whiplash trades - {whiplash} ({round(whiplash/total_trades,4)*100}%)')
    print("")
    longest_wins=0
    wins_count=0
    for i,row in results.iterrows():
        if row['win_loss']=="WIN":
            wins_count+=1
        else:
            if wins_count>longest_wins:
                longest_wins=wins_count
            wins_count=0
    print(f'Longest Wining streak: {longest_wins}')
    longest_loss=0
    loss_count=0
    for i,row in results.iterrows():
        if row['win_loss']=="LOSS":
            loss_count+=1
        else:
            if loss_count>longest_loss:
                longest_loss=loss_count
            loss_count=0
    print(f'Longest Losing streak: {longest_loss}')

In [18]:
#With this function we can print out some statistic from our backtest results
show_results(results)

End result is 901.04, loss of 98.96 (9.9%)

Total trade count is 49477
WIN trades - 21126(42.7%)
    Buy trades - 12436 (58.87%)
    Sell trades - 8690 (41.13%)
LOSS trades - 23134(46.76%)
    Buy trades - 10122 (43.75%)
    Sell trades - 13012 (56.25%)

Whiplash trades - 5217 (10.54%)

Longest Wining streak: 25
Longest Losing streak: 20


In [ ]:
# We can see that this strategy would not be profitable. But now we can change period from 10 previous candles to another and see how that would change our results.
# Also we can change another variables in our trading strategy like position sizing, exits, risk to reward ratio and others.